In [3]:
# ================== STEP 1: Install Required Packages ==================
# Set non-interactive Matplotlib backend
import os
os.environ['MPLBACKEND'] = 'Agg'
os.environ['MATPLOTLIBRC'] = '/dev/null'
import matplotlib
matplotlib.use('Agg')

# Install required packages
!pip install ultralytics --quiet
!pip install filterpy --quiet

# Download sort.py
!wget -O sort.py https://raw.githubusercontent.com/abewley/sort/master/sort.py

# Clean up sort.py to remove GUI/matplotlib dependencies
!sed -i '/matplotlib/d' sort.py
!sed -i '/plotting/d' sort.py
!sed -i '/TkAgg/d' sort.py
!sed -i '/skimage/d' sort.py

!pip install -q ultralytics
!pip install -q opencv-python-headless scipy


# ================== STEP 2: Imports ==================
import os
import cv2
import numpy as np
from ultralytics import YOLO
from sort import Sort
from scipy.spatial.distance import euclidean
from IPython.display import FileLink

# ================== STEP 3: File Paths ==================
MODEL_PATH = '/content/best.pt'
VIDEO_PATH = '/content/15sec_input_720p.mp4'
OUTPUT_PATH = '/content/output_tracked.mp4'

# ================== STEP 4: Load Model ==================
model = YOLO(MODEL_PATH)

# ================== STEP 5: Load Video ==================
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise Exception("Could not open video")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# ================== STEP 6: Tracking Variables ==================
tracker = Sort(max_age=120, min_hits=1, iou_threshold=0.03)

player_ids = {}  # track_id -> player_id
inactive_tracks = {}  # player_id -> (center_x, center_y, width, height, last_seen_frame)
last_track_boxes = {}

next_id = 1
MAX_PLAYERS = 25

def match_reentering_player(track, inactive_tracks, frame_count, max_gap=120):
    x1, y1, x2, y2, track_id = track
    cx, cy = (x1+x2)/2, (y1+y2)/2
    w, h = x2 - x1, y2 - y1

    best_match_id = None
    min_score = float('inf')

    for pid, (px, py, pw, ph, last_frame) in inactive_tracks.items():
        if frame_count - last_frame <= max_gap:
            dist = euclidean((cx, cy), (px, py))
            size_diff = abs(w - pw) + abs(h - ph)
            score = dist + size_diff * 0.5
            if score < min_score and score < 300:
                best_match_id = pid
                min_score = score

    return best_match_id

# ================== STEP 7: Processing Loop ==================
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # === Detection ===
    results = model(frame, conf=0.3, iou=0.45)
    detections = []
    for det in results[0].boxes:
        class_id = int(det.cls[0])
        name = model.names[class_id]
        if name.lower() == 'player':
            x1, y1, x2, y2 = map(float, det.xyxy[0])
            conf = float(det.conf[0])
            detections.append([x1, y1, x2, y2, conf])

    detections = np.array(detections, dtype=np.float32) if detections else np.empty((0, 5), dtype=np.float32)

    # === Tracking ===
    tracked = tracker.update(detections)

    # === Track management ===
    active_ids = set()
    for track in tracked:
        x1, y1, x2, y2, track_id = map(int, track)
        last_track_boxes[str(track_id)] = (x1, y1, x2, y2)
        active_ids.add(str(track_id))

    # Mark inactive
    for track_id in list(player_ids.keys()):
        if track_id not in active_ids:
            pid = player_ids[track_id]
            if track_id in last_track_boxes:
                x1, y1, x2, y2 = last_track_boxes[track_id]
                cx, cy = (x1+x2)/2, (y1+y2)/2
                w, h = x2 - x1, y2 - y1
                inactive_tracks[pid] = (cx, cy, w, h, frame_count)
                del last_track_boxes[track_id]

    # Assign or reassign player IDs
    for track in tracked:
        x1, y1, x2, y2, track_id = map(int, track)
        track_id = str(track_id)

        if track_id not in player_ids:
            matched_id = match_reentering_player(track, inactive_tracks, frame_count)
            if matched_id:
                player_ids[track_id] = matched_id
                del inactive_tracks[matched_id]
            elif next_id <= MAX_PLAYERS:
                player_ids[track_id] = next_id
                next_id += 1

        pid = player_ids.get(track_id, -1)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'Player {pid}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    out.write(frame)
    frame_count += 1

# ================== STEP 8: Cleanup ==================
cap.release()
out.release()

print(f"✅ Done. Video saved to {OUTPUT_PATH}")
FileLink(OUTPUT_PATH)


--2025-07-13 15:22:59--  https://raw.githubusercontent.com/abewley/sort/master/sort.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11739 (11K) [text/plain]
Saving to: ‘sort.py’

sort.py             100%[===================>]  11.46K  --.-KB/s    in 0s      

2025-07-13 15:22:59 (90.3 MB/s) - ‘sort.py’ saved [11739/11739]


0: 384x640 1 ball, 16 players, 2 referees, 2046.1ms
Speed: 3.0ms preprocess, 2046.1ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 1861.8ms
Speed: 2.1ms preprocess, 1861.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 15 players, 2 referees, 1870.4ms
Speed: 1.9ms preprocess, 1870.4ms inference, 0.8ms postprocess per image at shape (1, 3, 38

/content/output_tracked.mp4